# Video summary and visual question answering

In [ ]:
import ammico

Currently this module supports only video summarization, but it will be updated in the nearest future

In [ ]:
video_dict = ammico.find_videos(
    path=str("/insert/your/path/here/"),  # path to the folder with images
    limit=-1,  # -1 means no limit on the number of files, by default it is set to 20
)

In [ ]:
model = ammico.MultimodalSummaryModel()

In [ ]:
vid_summary_model = ammico.VideoSummaryDetector(summary_model=model, subdict=video_dict)

In [ ]:
summary_dict = vid_summary_model.analyse_video()

In [ ]:
summary_dict["summary"]